In [1]:
import time
import copy
import random

import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.cluster import KMeans

from graph_tree_list_module import *


In [2]:
raw = pd.read_csv(r'../Data/train.csv')
# print(raw)

random_seed = 42

# print(list(raw.columns)[:])

In [3]:
data = raw.iloc[:,2:-1]
data = data.fillna(0)
data = data[:5000]
print(data.shape)
scaler = MinMaxScaler()
normalized_data = scaler.fit_transform(data)

# label
y = raw.iloc[:,-1]
# print(y)

col_name = list(raw.columns)[2:-1]
print(col_name)
# print(normalized_data)

(5000, 14)
['Popularity', 'danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo', 'duration_in min/ms', 'time_signature']


In [4]:
data_min = scaler.data_min_
data_max = scaler.data_max_

# print(data_min, data_max)

In [5]:
def test_query_generator():
    return [random.uniform(data_min[i], data_max[i]) for i in range(len(data_max))]

In [6]:
print(test_query_generator())

[np.float64(14.399086231288727), np.float64(0.22093543399906762), np.float64(0.23431345107987042), np.float64(10.73145911840003), np.float64(-19.98722174628944), np.float64(0.23266360908837658), np.float64(0.5861075217932307), np.float64(0.01187831212614663), np.float64(0.10776350978885095), np.float64(0.866001609933342), np.float64(0.2792605759934712), np.float64(132.06342086976278), np.float64(922788.1271383524), np.float64(3.5634036148816666)]


In [7]:
import time

class Timer:
    def __enter__(self):
        self.start_time = time.perf_counter()
        return self

    def __exit__(self, *args):
        self.end_time = time.perf_counter()
        self.elapsed_time = self.end_time - self.start_time
        print(f"Elapsed time: {self.elapsed_time} seconds")

In [8]:
test_tree = graph_tree(attr_name = col_name, X = normalized_data, X_maxmin=[scaler.data_max_,scaler.data_min_])
# test_tree.show_root()
with Timer():
    test_tree.build_tree(min_threshold_for_clustering=70)

cur node 0
optimal K 2
cur node 1
optimal K 2
cur node 2
optimal K 2
cur node 3
optimal K 3
cur node 4
optimal K 2
cur node 5
optimal K 3
cur node 6
optimal K 2
cur node 7
optimal K 2
cur node 8
optimal K 2
cur node 9
optimal K 2
cur node 10
optimal K 2
cur node 11
optimal K 2
cur node 12
optimal K 2
cur node 13
optimal K 2
cur node 14
optimal K 3
cur node 15
optimal K 2
cur node 16
optimal K 2
cur node 17
optimal K 2
cur node 18
optimal K 4
cur node 19
optimal K 2
cur node 20
optimal K 5
cur node 21
optimal K 2
cur node 22
optimal K 2
cur node 23
optimal K 2
cur node 24
optimal K 3
cur node 25
optimal K 2
cur node 26
optimal K 2
cur node 27
optimal K 2
cur node 28
optimal K 9
cur node 29
optimal K 2
cur node 30
optimal K 6
cur node 31
cur node 32
cur node 33
cur node 34
optimal K 2
cur node 35
optimal K 2
cur node 36
cur node 37
cur node 38
optimal K 2
cur node 39
optimal K 2
cur node 40
cur node 41
optimal K 5
cur node 42
optimal K 3
cur node 43
optimal K 3
cur node 44
optimal K 3
cu

In [9]:
# import pickle

# with open("graph_tree.pkl", 'wb') as f:
#     pickle.dump(test_tree, f)

In [10]:
seaching = test_query_generator()
with Timer():
    cur = test_tree.search(seaching)
print(cur.id)

Elapsed time: 0.0036665000079665333 seconds
81


In [11]:
with Timer():
    res = test_tree.get_sorted_node_within_lv(test_tree.nodes[19].center, test_tree.nodes[1], lv=3)
print(res.keys())
print(res["sorted_cos_sim"])
print(res["euclide_dist"])
print([n.id for n in res["sorted_nodes"]])


generate cos sim, ecul dist, and nodes sorted by cos sim
[1]
[3, 4]
[7, 8, 9, 10, 11]
[17, 18, 19, 20, 21, 22, 23, 24, 25, 26]
Elapsed time: 0.0006804999866290018 seconds
dict_keys(['sorted_cos_sim', 'euclide_dist', 'sorted_nodes'])
[np.float64(-0.7196024799360232), np.float64(-0.5166752695007447), np.float64(-0.46808632831274066), np.float64(-0.43762710793731385), np.float64(-0.154108508586458), np.float64(-0.10508860493442983), np.float64(0.0816651754899626), np.float64(0.32040194594876625), np.float64(0.4078552582944558)]
[np.float64(0.9245285781833025), np.float64(1.4359582404856008), np.float64(1.0966716374419048), np.float64(0.7395003521569773), np.float64(0.8601384239958914), np.float64(0.7926743656074626), np.float64(0.5737491642933192), np.float64(0.7097288419947511), np.float64(0.45412132367302266)]
[25, 23, 24, 18, 22, 26, 17, 21, 20]


In [12]:
print(calculation.l2(test_tree.nodes[9].center, test_tree.nodes[12].center))

1.296546715446036


In [17]:
a = test_tree.lv_start
print(a)
print([a[i+1]-a[i] for i in range(len(a)-1)])

[0, 1, 3, 7, 17, 38, 87, 172, 242, 256]
[1, 2, 4, 10, 21, 49, 85, 70, 14]


In [13]:
test_tree.print_tree(test_tree.nodes[0])

└── 0
    ├── 1
    │   ├── 3
    │   │   ├── 7
    │   │   │   ├── 17
    │   │   │   │   ├── 38
    │   │   │   │   │   ├── 87
    │   │   │   │   │   │   ├── 172
    │   │   │   │   │   │   ├── 173
    │   │   │   │   │   │   └── 174
    │   │   │   │   │   └── 88
    │   │   │   │   │       ├── 175
    │   │   │   │   │       └── 176
    │   │   │   │   └── 39
    │   │   │   │       ├── 89
    │   │   │   │       │   ├── 177
    │   │   │   │       │   │   ├── 242
    │   │   │   │       │   │   ├── 243
    │   │   │   │       │   │   ├── 244
    │   │   │   │       │   │   ├── 245
    │   │   │   │       │   │   ├── 246
    │   │   │   │       │   │   ├── 247
    │   │   │   │       │   │   ├── 248
    │   │   │   │       │   │   └── 249
    │   │   │   │       │   └── 178
    │   │   │   │       │       ├── 250
    │   │   │   │       │       ├── 251
    │   │   │   │       │       └── 252
    │   │   │   │       └── 90
    │   │   │   │           ├── 179
    │   │   │   │      

In [14]:
print(test_tree.nodes[0].center)

[0.43906465 0.52589225 0.66144496 0.47741818 0.7539839  0.639
 0.06208811 0.25001819 0.13598254 0.1839972  0.47853415 0.48415781
 0.14202648 0.7291    ]


In [15]:
res = test_tree.reflection_search(np.mean(normalized_data[:100], axis=0), node=test_tree.nodes[11], degree=2)

AttributeError: 'graph_tree' object has no attribute 'reflection_search'

In [15]:
res

[[<graph_tree_list_module.Node at 0x24a0919b800>],
 array([0.45153535, 0.52339993, 0.68550112, 0.49472727, 0.76952637,
        0.561     , 0.06974494, 0.27134241, 0.13702421, 0.19445554,
        0.47458816, 0.49446303, 0.19104194, 0.73775   ]),
 [np.int64(934),
  np.int64(461),
  np.int64(767),
  np.int64(1807),
  np.int64(1065),
  np.int64(915),
  np.int64(107),
  np.int64(1006),
  np.int64(1318),
  np.int64(803)]]

In [16]:
from collections import Counter

"""
create a new index (something like tf-idf)
data reduction rate
vs
accurate
"""
def score(search_data, acc, smoothing = 0.001, lam = 0.5):
    return lam*(acc/100)+(1-lam)*(1-(search_data/(1000+smoothing)))

my_score = []
normal_kmeans_score = []

tmp = [i*100 for i in range(10)]
for i in tmp:
    b = i
    u = i+100
    # print(normalized_data[b:u])
    print(f"-- mine result{i}")
    seaching = np.mean(normalized_data[b:u], axis=0)
    result = test_tree.search(seaching)
    
    print("search number: ", len(result.index))
    right = np.where((result.index>=b) & (result.index<u))[0] # return index
    print("hitted right result:", len(right))
    my_score.append(score(len(result.index), len(right)))

    ############
    print("-- kmeans result")
    checking = KMeans(n_clusters=10, random_state=0).fit(normalized_data)

    counts1 = Counter(checking.labels_)
    print(counts1)

    counts2 = Counter(checking.labels_[b:u])
    print(counts2)
    belongs_group = counts2.most_common()[0][0]
    print("belongs to group: ", belongs_group, "\n")
    # print(counts1.get(belongs_group), counts2.get(belongs_group))
    normal_kmeans_score.append(score(counts1.get(belongs_group), counts2.get(belongs_group)))

    

-- mine result0
search number:  181
hitted right result: 12
-- kmeans result
Counter({np.int32(4): 410, np.int32(2): 271, np.int32(6): 261, np.int32(1): 249, np.int32(8): 224, np.int32(0): 158, np.int32(5): 145, np.int32(7): 115, np.int32(3): 90, np.int32(9): 77})
Counter({np.int32(2): 20, np.int32(4): 20, np.int32(6): 14, np.int32(1): 12, np.int32(8): 9, np.int32(5): 7, np.int32(0): 6, np.int32(3): 6, np.int32(7): 5, np.int32(9): 1})
belongs to group:  2 

-- mine result100
search number:  2000
hitted right result: 100
-- kmeans result
Counter({np.int32(4): 410, np.int32(2): 271, np.int32(6): 261, np.int32(1): 249, np.int32(8): 224, np.int32(0): 158, np.int32(5): 145, np.int32(7): 115, np.int32(3): 90, np.int32(9): 77})
Counter({np.int32(4): 22, np.int32(6): 11, np.int32(1): 11, np.int32(2): 11, np.int32(5): 10, np.int32(0): 10, np.int32(7): 8, np.int32(9): 7, np.int32(8): 7, np.int32(3): 3})
belongs to group:  4 

-- mine result200
search number:  2000
hitted right result: 100
-- kme

In [17]:
print(my_score)
print(normal_kmeans_score)

print(sum(my_score))
print(sum(normal_kmeans_score))

[0.4695000904999095, 9.99998999939855e-07, 9.99998999939855e-07, 9.99998999939855e-07, 9.99998999939855e-07, 0.4095001804998195, 9.99998999939855e-07, 9.99998999939855e-07, 9.99998999939855e-07, 9.99998999939855e-07]
[0.46450013549986446, 0.40500020499979494, 0.37000020499979497, 0.41500020499979495, 0.40000020499979494, 0.43000020499979497, 0.3850002049997949, 0.41500020499979495, 0.42500020499979496, 0.4645001304998695]
0.8790082709917285
4.174001905998094
